In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from collections import deque
import random
import matplotlib.pyplot as plt
import os
import pickle
import json
import time
import traci
import sys
import gym
import datetime

In [3]:
df_traffic_lights=pd.read_csv("C:/Users/Pushp Raj Choudhary/OneDrive/Documents/Coding/Python/Traffic_Simulator/traffic_lights_extracted.csv")

In [4]:
df_traffic_lights.tail(5)

,Timestep,Traffic_Light_ID,State,Phase_Duration
11095,3698,cluster_11502623497_1177160337,GGrrr,42.000
11096,3698,cluster_664446535_8507351666,rryyy,3.000
11097,3699,cluster_10279701148_11630151856_663826006,GG,0.001
11098,3699,cluster_11502623497_1177160337,GGrrr,42.000
11099,3699,cluster_664446535_8507351666,GGrrr,42.000


In [5]:
df_traffic_lights = df_traffic_lights[df_traffic_lights["Traffic_Light_ID"] != "cluster_10279701148_11630151856_663826006"]   # Remove this traffic light as it is not present (or giving false sense of presence [giving green state always]) in the simulation
df_traffic_lights.tail(5)

,Timestep,Traffic_Light_ID,State,Phase_Duration
11093,3697,cluster_664446535_8507351666,rryyy,3.0
11095,3698,cluster_11502623497_1177160337,GGrrr,42.0
11096,3698,cluster_664446535_8507351666,rryyy,3.0
11098,3699,cluster_11502623497_1177160337,GGrrr,42.0
11099,3699,cluster_664446535_8507351666,GGrrr,42.0


In [6]:
df_traffic_lights['Traffic_Light_1_2'] = df_traffic_lights['State'].apply(lambda x: x[:2])  # First two traffic light states (Same path but diff. direction)
df_traffic_lights['Traffic_Light_3_4'] = df_traffic_lights['State'].apply(lambda x: x[2:4])  # Second two traffic light states (Same path but diff. direction)
df_traffic_lights['Zebra_Crossing_Light'] = df_traffic_lights['State'].apply(lambda x: x[4])  # Zebra crossing state (Same path but diff. direction)

In [25]:
df_traffic_lights.tail(5)
df_traffic_lights.to_csv("C:/Users/Pushp Raj Choudhary/Documents/Coding/Python/Traffic_Simulator/traffic_lights_extracted_with_differentiation.csv", index=False)

In [28]:
df_combined = pd.read_csv("C:/Users/Pushp Raj Choudhary/Documents/Coding/Python/Traffic_Simulator/combined_sumo_data2.csv")
df_combined.tail(5)

,Timestep,Vehicle_ID,Speed,Lane_ID,Edge_ID,X_Position,Y_Position,Acceleration,Angle,Depart_Time,...,Waiting_Time,Time_Loss,Distance_Traveled,CO2_Emission,Fuel_Consumption,NOx_Emission,PMx_Emission,Traffic_Light_ID,State,Phase_Duration
769473,3739.0,veh409,5.74,:10943498141_3_0,Unknown,241.80,220.00,0.0,101.93,3577.0,...,0.0,35.01,1902.73,0.00,0.00,0.00,0.00,0,0,0.0
769474,3740.0,veh409,5.64,:10943498141_3_0,Unknown,246.05,223.16,0.0,50.33,3577.0,...,0.0,35.01,1902.73,1988.18,634.17,0.75,0.02,0,0,0.0
769475,3741.0,veh409,6.56,-101966385#4_0,Unknown,248.18,229.36,0.0,17.62,3577.0,...,0.0,35.01,1902.73,3687.26,1176.08,1.53,0.06,0,0,0.0
769476,3742.0,veh409,8.45,-101966385#4_0,Unknown,250.74,237.42,0.0,17.62,3577.0,...,0.0,35.01,1902.73,6262.04,1997.30,2.69,0.12,0,0,0.0
769477,3743.0,veh409,10.55,-101966385#4_0,Unknown,253.93,247.48,0.0,17.62,3577.0,...,0.0,35.01,1902.73,7905.93,2521.61,3.41,0.16,0,0,0.0


##### Last few rows after 3700 timet steps are not recorded for the traffic light so we will use the 3700 timesteps for training and execution

In [30]:
df_combined = df_combined[df_combined['Timestep'] <= 3699]
df_combined.tail(5)

,Timestep,Vehicle_ID,Speed,Lane_ID,Edge_ID,X_Position,Y_Position,Acceleration,Angle,Depart_Time,...,Waiting_Time,Time_Loss,Distance_Traveled,CO2_Emission,Fuel_Consumption,NOx_Emission,PMx_Emission,Traffic_Light_ID,State,Phase_Duration
769429,3698.0,veh411,11.10,103050330#0_0,Unknown,893.40,505.79,0.0,109.04,3595.0,...,1.0,32.16,1268.25,5908.83,1884.64,2.46,0.11,cluster_664446535_8507351666,rryyy,3.0
769430,3699.0,veh409,22.40,:1190100061_1_0,Unknown,363.67,406.63,0.0,287.97,3577.0,...,0.0,35.01,1902.73,5031.86,1604.93,1.77,0.08,cluster_11502623497_1177160337,GGrrr,42.0
769431,3699.0,veh409,22.40,:1190100061_1_0,Unknown,363.67,406.63,0.0,287.97,3577.0,...,0.0,35.01,1902.73,5031.86,1604.93,1.77,0.08,cluster_664446535_8507351666,GGrrr,42.0
769432,3699.0,veh411,13.57,103050330#0_0,Unknown,906.23,501.36,0.0,109.04,3595.0,...,1.0,32.16,1268.25,11054.47,3525.82,4.82,0.24,cluster_11502623497_1177160337,GGrrr,42.0
769433,3699.0,veh411,13.57,103050330#0_0,Unknown,906.23,501.36,0.0,109.04,3595.0,...,1.0,32.16,1268.25,11054.47,3525.82,4.82,0.24,cluster_664446535_8507351666,GGrrr,42.0


In [31]:
df_combined=df_combined.drop(columns=['Traffic_Light_ID', 'State', 'Phase_Duration'])
df_combined = pd.merge(df_combined, df_traffic_lights, on=['Timestep'], how='left')
df_combined.tail(5)

,Timestep,Vehicle_ID,Speed,Lane_ID,Edge_ID,X_Position,Y_Position,Acceleration,Angle,Depart_Time,...,CO2_Emission,Fuel_Consumption,NOx_Emission,PMx_Emission,Traffic_Light_ID,State,Phase_Duration,Traffic_Light_1_2,Traffic_Light_3_4,Zebra_Crossing_Light
1538863,3699.0,veh409,22.40,:1190100061_1_0,Unknown,363.67,406.63,0.0,287.97,3577.0,...,5031.86,1604.93,1.77,0.08,cluster_664446535_8507351666,GGrrr,42.0,GG,rr,r
1538864,3699.0,veh411,13.57,103050330#0_0,Unknown,906.23,501.36,0.0,109.04,3595.0,...,11054.47,3525.82,4.82,0.24,cluster_11502623497_1177160337,GGrrr,42.0,GG,rr,r
1538865,3699.0,veh411,13.57,103050330#0_0,Unknown,906.23,501.36,0.0,109.04,3595.0,...,11054.47,3525.82,4.82,0.24,cluster_664446535_8507351666,GGrrr,42.0,GG,rr,r
1538866,3699.0,veh411,13.57,103050330#0_0,Unknown,906.23,501.36,0.0,109.04,3595.0,...,11054.47,3525.82,4.82,0.24,cluster_11502623497_1177160337,GGrrr,42.0,GG,rr,r
1538867,3699.0,veh411,13.57,103050330#0_0,Unknown,906.23,501.36,0.0,109.04,3595.0,...,11054.47,3525.82,4.82,0.24,cluster_664446535_8507351666,GGrrr,42.0,GG,rr,r


In [32]:
# Directories for checkpoints and results
os.makedirs("checkpoints", exist_ok=True)
os.makedirs("results", exist_ok=True)
os.makedirs("models", exist_ok=True)
os.makedirs("logs", exist_ok=True)

In [33]:
def load_data():
    try:
        print("Loading combined data...")
        combined_data = df_combined.copy()  # Use the modified DataFrame
        print(f"Data loaded successfully with {len(combined_data)} records")
        # Save a backup of the original data
        combined_data.to_csv("checkpoints/original_data_backup.csv", index=False)
        return combined_data
    except Exception as e:
        print(f"Error loading data: {e}")
        return None

In [34]:
def init_sumo(sumocfg_path):
    try:
        print("Initializing SUMO environment...")
        sumo_cmd = ["sumo", "-c", sumocfg_path, "--start"]
        traci.start(sumo_cmd)
        print("SUMO environment initialized successfully")
        return True
    except Exception as e:
        print(f"Error initializing SUMO: {e}")
        return False

In [54]:
# State representation
def get_state(traffic_data, current_timestep):
    """Extract state features from traffic data at the current timestep"""
    try:
        # Filter data for current timestep
        current_data = traffic_data[traffic_data["Timestep"] == current_timestep]
        
        if current_data.empty:
            print(f"No data found for timestep {current_timestep}")
            return None
        
        # Load lane groups mapping if not provided
        if lane_groups_mapping is None:
            lane_groups_mapping = pd.read_csv("C:/Users/Pushp Raj Choudhary/Documents/Coding/Python/Traffic_Simulator/lane_groups.csv")
            # Create a dictionary mapping lane IDs to their group
            lane_to_group = {}
            for col in lane_groups_mapping.columns:
                group = col
                for lane in lane_groups_mapping[col].dropna().values:
                    lane_to_group[lane] = group
        
        # Create Lane_Group using the mapping
        current_data.loc[:, "Lane_Group"] = current_data["Lane_ID"].map(lane_to_group)
        # Fill any unmapped lanes with "Other"
        current_data.loc[:, "Lane_Group"] = current_data["Lane_Group"].fillna("Other")
        
        # Extract vehicle density per lane group
        lane_density = current_data.groupby("Lane_Group")["Vehicle_ID"].count().to_dict()
        
        # Extract average speed per lane group
        avg_speed = current_data.groupby("Lane_Group")["Speed"].mean().to_dict()
        
        # Extract waiting time
        avg_waiting_time = current_data["Waiting_Time"].mean()
        
        # Extract traffic light states
        tl_states = current_data[["Traffic_Light_1_2", "Traffic_Light_3_4", "Zebra_Crossing_Light"]].drop_duplicates().values.tolist()
        
        # Combine features into state vector
        state_features = []
        
        # Add lane density features
        for lane_group in sorted(lane_density.keys()):
            state_features.append(lane_density.get(lane_group, 0))
        
        # Add speed features
        for lane_group in sorted(avg_speed.keys()):
            state_features.append(avg_speed.get(lane_group, 0))
        
        # Add waiting time
        state_features.append(avg_waiting_time)
        
        # Add traffic light features (one-hot encode the traffic light states)
        for tl in tl_states:
            # Encode Traffic_Light_1_2 (one-hot encoding for each state)
            if 'GG' in tl[0]:  # Green state
                state_features.extend([1, 0, 0])
            elif 'yy' in tl[0]:  # Yellow state
                state_features.extend([0, 1, 0])
            else:  # Red state
                state_features.extend([0, 0, 1])
            
            # Encode Traffic_Light_3_4 (one-hot encoding for each state)
            if 'GG' in tl[1]:  # Green state
                state_features.extend([1, 0, 0])
            elif 'yy' in tl[1]:  # Yellow state
                state_features.extend([0, 1, 0])
            else:  # Red state
                state_features.extend([0, 0, 1])
            
            # Encode Zebra_Crossing_Light (one-hot encoding for each state)
            if 'G' or 'g' in tl[2]:  # Green state
                state_features.extend([1, 0, 0])
            elif 'y' in tl[2]:  # Yellow state
                state_features.extend([0, 1, 0])
            else:  # Red state
                state_features.extend([0, 0, 1])
        
        # Save state representation for debugging
        with open(f"checkpoints/state_representation_{current_timestep}.pkl", "wb") as f:
            pickle.dump(state_features, f)            
        return np.array(state_features, dtype=np.float32)
    except Exception as e:
        print(f"Error in get_state: {e}")
        return None

In [42]:
# Action Implementation in SUMO using TraCI API
def take_action(action, traffic_light_ids):
    """
    Implement the selected action in the SUMO environment
    
    Actions:
    0: Maintain current phase
    1: Switch state of Traffic_Light_1_2
    2: Switch state of Traffic_Light_3_4
    3: Switch Zebra_Crossing_Light
    4: Extend green phase for Traffic_Light_1_2
    5: Reduce green phase for Traffic_Light_1_2
    6: Increase yellow phase for Traffic_Light_1_2
    7: Decrease yellow phase for Traffic_Light_1_2
    """
    try:
        action_log = {}
        for tl_id in traffic_light_ids:
            prev_phase = traci.trafficlight.getPhase(tl_id)
            prev_duration = traci.trafficlight.getPhaseDuration(tl_id)
            
            if action == 0:
                # Action 0: Maintain current phase
                pass
            elif action == 1:
                # Action 1: Switch state of Traffic_Light_1_2
                next_phase = (prev_phase + 1) % traci.trafficlight.getPhaseCount(tl_id)
                traci.trafficlight.setPhase(tl_id, next_phase)
            elif action == 2:
                # Action 2: Switch state of Traffic_Light_3_4
                next_phase = (prev_phase + 1) % traci.trafficlight.getPhaseCount(tl_id)
                traci.trafficlight.setPhase(tl_id, next_phase)
            elif action == 3:
                # Action 3: Switch Zebra Crossing Light
                next_phase = (prev_phase + 1) % traci.trafficlight.getPhaseCount(tl_id)
                traci.trafficlight.setPhase(tl_id, next_phase)
            elif action == 4:
                # Action 4: Extend green phase for Traffic_Light_1_2
                current_duration = traci.trafficlight.getPhaseDuration(tl_id)
                traci.trafficlight.setPhaseDuration(tl_id, current_duration + 5)
            elif action == 5:
                # Action 5: Reduce green phase for Traffic_Light_1_2
                current_duration = traci.trafficlight.getPhaseDuration(tl_id)
                if current_duration > 10:
                    traci.trafficlight.setPhaseDuration(tl_id, current_duration - 5)
            elif action == 6:
                # Action 6: Increase yellow phase for Traffic_Light_1_2
                current_duration = traci.trafficlight.getPhaseDuration(tl_id)
                traci.trafficlight.setPhaseDuration(tl_id, current_duration + 5)
            elif action == 7:
                # Action 7: Decrease yellow phase for Traffic_Light_1_2
                current_duration = traci.trafficlight.getPhaseDuration(tl_id)
                if current_duration > 5:
                    traci.trafficlight.setPhaseDuration(tl_id, current_duration - 5)
            
            # Log the action for debugging
            action_log[tl_id] = {
                "prev_phase": prev_phase,
                "prev_duration": prev_duration,
                "new_phase": traci.trafficlight.getPhase(tl_id),
                "new_duration": traci.trafficlight.getPhaseDuration(tl_id),
                "action_taken": action
            }
        
        # Save action log for debugging
        with open(f"logs/action_log_{time.time()}.json", "w") as f:
            json.dump(action_log, f)         
        return True
    except Exception as e:
        print(f"Error in take_action: {e}")
        return False

In [43]:
class DQN(nn.Module):
    def __init__(self, state_size, action_size):
        super(DQN, self).__init__()
        self.fc1 = nn.Linear(state_size, 128)
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, action_size)
        
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        return self.fc3(x)
    
    def save(self, path):
        torch.save(self.state_dict(), path)
        
    def load(self, path):
        self.load_state_dict(torch.load(path))

In [44]:
class ReplayBuffer:
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)
    
    def add(self, state, action, reward, next_state, done):
        self.buffer.append((state, action, reward, next_state, done))
    
    def sample(self, batch_size):
        return random.sample(self.buffer, batch_size)
    
    def __len__(self):
        return len(self.buffer)
    
    def save(self, path):
        with open(path, 'wb') as f:
            pickle.dump(self.buffer, f)
    
    def load(self, path):
        with open(path, 'rb') as f:
            self.buffer = pickle.load(f)

In [46]:
class DQNAgent:
    def __init__(self, state_size, action_size, learning_rate=0.001, gamma=0.95, 
                 epsilon=1.0, epsilon_decay=0.995, epsilon_min=0.01, 
                 buffer_size=10000, batch_size=32):
        self.state_size = state_size
        self.action_size = action_size
        self.gamma = gamma  # discount factor
        self.epsilon = epsilon  # exploration rate
        self.epsilon_decay = epsilon_decay
        self.epsilon_min = epsilon_min
        self.batch_size = batch_size
        
        # Initialize Q-networks (online and target)
        self.q_network = DQN(state_size, action_size)
        self.target_network = DQN(state_size, action_size)
        self.target_network.load_state_dict(self.q_network.state_dict())
        
        # Initialize optimizer
        self.optimizer = optim.Adam(self.q_network.parameters(), lr=learning_rate)
        
        # Initialize replay buffer
        self.memory = ReplayBuffer(buffer_size)
        
        # Initialize loss function
        self.criterion = nn.MSELoss()
        
        # Training metrics
        self.loss_history = []
        self.reward_history = []
        self.epsilon_history = []
    
    def update_target_network(self):
        """Update target network with weights from online network"""
        self.target_network.load_state_dict(self.q_network.state_dict())
    
    def select_action(self, state):
        """Select action using epsilon-greedy policy"""
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        
        state_tensor = torch.FloatTensor(state).unsqueeze(0)
        with torch.no_grad():
            q_values = self.q_network(state_tensor)
        return q_values.argmax().item()
    
    def store_experience(self, state, action, reward, next_state, done):
        """Store experience in replay buffer"""
        self.memory.add(state, action, reward, next_state, done)
    
    def train(self):
        """Train the agent using experiences from replay buffer"""
        if len(self.memory) < self.batch_size:
            return 0  # Return 0 loss if not enough samples
        
        # Sample batch from replay buffer
        batch = self.memory.sample(self.batch_size)
        states, actions, rewards, next_states, dones = zip(*batch)
        
        # Convert to tensors
        states = torch.FloatTensor(np.array(states))
        actions = torch.LongTensor(actions).unsqueeze(1)
        rewards = torch.FloatTensor(rewards)
        next_states = torch.FloatTensor(np.array(next_states))
        dones = torch.FloatTensor(dones)
        
        # Compute current Q values
        current_q_values = self.q_network(states).gather(1, actions).squeeze()
        
        # Compute target Q values
        with torch.no_grad():
            max_next_q_values = self.target_network(next_states).max(1)[0]
            target_q_values = rewards + (1 - dones) * self.gamma * max_next_q_values
        
        # Compute loss
        loss = self.criterion(current_q_values, target_q_values)
        
        # Update online network
        self.optimizer.zero_grad()
        loss.backward()
        self.optimizer.step()
        
        # Record loss
        self.loss_history.append(loss.item())
        
        # Decay epsilon
        if self.epsilon > self.epsilon_min:
            self.epsilon *= self.epsilon_decay
            self.epsilon_history.append(self.epsilon)
        
        return loss.item()
    
    def save(self, episode):
        """Save agent state"""
        checkpoint = {
            'q_network_state': self.q_network.state_dict(),
            'target_network_state': self.target_network.state_dict(),
            'optimizer_state': self.optimizer.state_dict(),
            'epsilon': self.epsilon,
            'loss_history': self.loss_history,
            'reward_history': self.reward_history,
            'epsilon_history': self.epsilon_history
        }
        torch.save(checkpoint, f"checkpoints/agent_checkpoint_ep{episode}.pt")
        self.memory.save(f"checkpoints/replay_buffer_ep{episode}.pkl")
        
        # Save metrics separately for easy access
        np.save(f"results/loss_history_ep{episode}.npy", np.array(self.loss_history))
        np.save(f"results/reward_history_ep{episode}.npy", np.array(self.reward_history))
        np.save(f"results/epsilon_history_ep{episode}.npy", np.array(self.epsilon_history))
    
    def load(self, episode):
        """Load agent state"""
        try:
            checkpoint = torch.load(f"checkpoints/agent_checkpoint_ep{episode}.pt")
            self.q_network.load_state_dict(checkpoint['q_network_state'])
            self.target_network.load_state_dict(checkpoint['target_network_state'])
            self.optimizer.load_state_dict(checkpoint['optimizer_state'])
            self.epsilon = checkpoint['epsilon']
            self.loss_history = checkpoint['loss_history']
            self.reward_history = checkpoint['reward_history']
            self.epsilon_history = checkpoint['epsilon_history']
            
            # Load replay buffer
            self.memory.load(f"checkpoints/replay_buffer_ep{episode}.pkl")
            
            print(f"Successfully loaded checkpoint from episode {episode}")
            return True
        except Exception as e:
            print(f"Error loading checkpoint: {e}")
            return False

In [47]:
def calculate_reward(current_data, previous_data, action_taken):
    """Calculate reward based on traffic metrics"""
    try:
        # Initialize reward components dictionary for logging
        reward_components = {}
        
        # Calculate average waiting time difference
        current_waiting = current_data["Waiting_Time"].mean()
        previous_waiting = previous_data["Waiting_Time"].mean() if previous_data is not None else 0
        waiting_diff = previous_waiting - current_waiting
        reward_components["waiting_diff"] = waiting_diff
        
        # Calculate average speed difference
        current_speed = current_data["Speed"].mean()
        previous_speed = previous_data["Speed"].mean() if previous_data is not None else 0
        speed_diff = current_speed - previous_speed
        reward_components["speed_diff"] = speed_diff
        
        # Calculate emissions difference
        current_emissions = current_data["CO2_Emission"].mean()
        previous_emissions = previous_data["CO2_Emission"].mean() if previous_data is not None else 0
        emissions_diff = previous_emissions - current_emissions
        reward_components["emissions_diff"] = emissions_diff
        
        # Calculate queue length difference (using vehicle count as proxy)
        current_queue = current_data.groupby("Lane_Group")["Vehicle_ID"].count().sum()
        previous_queue = previous_data.groupby("Lane_Group")["Vehicle_ID"].count().sum() if previous_data is not None else 0
        queue_diff = previous_queue - current_queue
        reward_components["queue_diff"] = queue_diff
        
        # Apply penalties for signal changes to discourage frequent switching
        signal_change_penalty = -5 if action_taken > 0 else 0
        reward_components["signal_change_penalty"] = signal_change_penalty
        
        # Combine rewards with appropriate weights
        reward = (
            2.0 * waiting_diff +    # Prioritize waiting time reduction
            1.0 * speed_diff +      # Reward speed improvements
            0.5 * emissions_diff +  # Minor reward for emissions reduction
            1.5 * queue_diff +      # Reward queue length reduction
            signal_change_penalty   # Penalty for changing signals
        )
        
        reward_components["total_reward"] = reward
        
        # Log reward components
        with open(f"logs/reward_log_{time.time()}.json", "w") as f:
            json.dump(reward_components, f)
        
        return reward
    except Exception as e:
        print(f"Error in calculate_reward: {e}")
        return 0  # Default to neutral reward on error

In [48]:
def plot_training_progress(rewards, waiting_times, emissions, episode):
    """Plot and save training progress"""
    plt.figure(figsize=(15, 10))
    
    # Plot rewards
    plt.subplot(3, 1, 1)
    plt.plot(rewards, 'b-')
    plt.title(f'Rewards (Episode {episode})')
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.grid(True)
    
    # Plot waiting times
    plt.subplot(3, 1, 2)
    plt.plot(waiting_times, 'r-')
    plt.title('Average Waiting Time')
    plt.xlabel('Episode')
    plt.ylabel('Waiting Time (s)')
    plt.grid(True)
    
    # Plot emissions
    plt.subplot(3, 1, 3)
    plt.plot(emissions, 'g-')
    plt.title('Average CO2 Emissions')
    plt.xlabel('Episode')
    plt.ylabel('CO2 (g)')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(f'results/training_progress_ep{episode}.png')
    plt.close()

In [49]:
def train_agent_segment(agent, num_episodes, max_steps_per_episode, sumocfg_path, 
                combined_data, start_episode=0, end_episode=None, 
                checkpoint_interval=5, resume_from=None):
    """Train the DQN agent with checkpointing for a specific segment of episodes"""
    
    if end_episode is None:
        end_episode = start_episode + num_episodes
    
    print(f"Training segment from episode {start_episode} to {end_episode}")
    rewards_history = []
    waiting_time_history = []
    emissions_history = []
    
    # Create segment-specific log directory
    segment_log_dir = f"logs/segment_{start_episode}_{end_episode}"
    os.makedirs(segment_log_dir, exist_ok=True)
    
    # Log file for this segment
    log_file = open(f"{segment_log_dir}/training_log.txt", "w")
    log_file.write(f"Training segment from episode {start_episode} to {end_episode}\n")
    log_file.write(f"Started at: {datetime.datetime.now()}\n\n")
    
    # Resume from checkpoint if specified
    if resume_from is not None:
        if agent.load(resume_from):
            print(f"Resuming training from checkpoint at episode {resume_from}")
            log_file.write(f"Resuming from checkpoint at episode {resume_from}\n")
            
            # Load previous metrics if available
            try:
                rewards_history = list(np.load(f"results/reward_history_ep{resume_from}.npy"))
                waiting_time_history = list(np.load(f"results/waiting_time_history_ep{resume_from}.npy"))
                emissions_history = list(np.load(f"results/emissions_history_ep{resume_from}.npy"))
                print("Loaded previous metrics successfully")
                log_file.write("Loaded previous metrics successfully\n")
            except Exception as e:
                print(f"Could not load previous metrics: {e}")
                log_file.write(f"Could not load previous metrics: {e}\n")
    
    try:
        for episode in range(start_episode, end_episode):
            episode_start_time = time.time()
            print(f"Starting episode {episode+1}/{end_episode}")
            log_file.write(f"\nEpisode {episode+1}/{end_episode}\n")
            
            # Connect to SUMO
            try:
                init_sumo(sumocfg_path)
                log_file.write("SUMO initialized successfully\n")
            except Exception as e:
                print(f"Error initializing SUMO: {e}")
                log_file.write(f"Error initializing SUMO: {e}\n")
                continue
            
            # Get traffic light IDs
            traffic_light_ids = traci.trafficlight.getIDList()
            log_file.write(f"Traffic lights: {traffic_light_ids}\n")
            
            # Initialize episode variables
            total_reward = 0
            previous_data = None
            episode_losses = []
            episode_waiting_times = []
            episode_emissions = []
            
            for step in range(max_steps_per_episode):
                if step % 100 == 0:
                    print(f"Episode {episode+1}, Step {step}/{max_steps_per_episode}")
                
                # Get current timestep
                current_timestep = step
                
                # Get current state
                current_data = combined_data[combined_data["Timestep"] == current_timestep]
                if current_data.empty:
                    print(f"No data for timestep {current_timestep}, ending episode")
                    log_file.write(f"No data for timestep {current_timestep}, ending episode\n")
                    break
                    
                state = get_state(combined_data, current_timestep)
                if state is None:
                    print(f"Could not get state for timestep {current_timestep}, skipping step")
                    log_file.write(f"Could not get state for timestep {current_timestep}, skipping step\n")
                    continue
                
                # Select action
                action = agent.select_action(state)
                
                # Take action in environment
                action_success = take_action(action, traffic_light_ids)
                if not action_success:
                    print("Action failed, skipping step")
                    log_file.write("Action failed, skipping step\n")
                    continue
                
                # Advance simulation by one step
                traci.simulationStep()
                
                # Get next state
                next_timestep = current_timestep + 1
                next_data = combined_data[combined_data["Timestep"] == next_timestep]
                
                if next_data.empty:
                    done = True
                    print(f"Episode ended at step {step} (no more data)")
                    log_file.write(f"Episode ended at step {step} (no more data)\n")
                else:
                    done = False
                    next_state = get_state(combined_data, next_timestep)
                    if next_state is None:
                        print(f"Could not get next state for timestep {next_timestep}, ending step")
                        log_file.write(f"Could not get next state for timestep {next_timestep}, ending step\n")
                        continue
                
                # Calculate reward
                reward = calculate_reward(current_data, previous_data, action)
                total_reward += reward
                
                # Record metrics for this step
                current_waiting_time = current_data["Waiting_Time"].mean()
                current_emissions = current_data["CO2_Emission"].mean()
                episode_waiting_times.append(current_waiting_time)
                episode_emissions.append(current_emissions)
                
                if not done:
                    # Store experience
                    agent.store_experience(state, action, reward, next_state, done)
                    
                    # Train agent
                    loss = agent.train()
                    episode_losses.append(loss)
                    
                    # Update previous data
                    previous_data = current_data
                else:
                    break
            
            # Close SUMO connection
            traci.close()
            
            # Calculate episode metrics
            episode_duration = time.time() - episode_start_time
            avg_waiting_time = np.mean(episode_waiting_times) if episode_waiting_times else 0
            avg_emissions = np.mean(episode_emissions) if episode_emissions else 0
            avg_loss = np.mean(episode_losses) if episode_losses else 0
            
            # Record metrics
            rewards_history.append(total_reward)
            waiting_time_history.append(avg_waiting_time)
            emissions_history.append(avg_emissions)
            
            # Update target network every episode
            agent.update_target_network()
            
            # Save checkpoint at specified intervals
            if episode % checkpoint_interval == 0 or episode == end_episode - 1:
                agent.save(episode)
                
                # Save metrics
                np.save(f"results/reward_history_ep{episode}.npy", np.array(rewards_history))
                np.save(f"results/waiting_time_history_ep{episode}.npy", np.array(waiting_time_history))
                np.save(f"results/emissions_history_ep{episode}.npy", np.array(emissions_history))
                
                # Plot and save current progress
                plot_training_progress(rewards_history, waiting_time_history, emissions_history, episode)
                
                print(f"Checkpoint saved at episode {episode}")
                log_file.write(f"Checkpoint saved at episode {episode}\n")
            
            # Print episode summary
            print(f"Episode {episode+1}/{end_episode}, Duration: {episode_duration:.2f}s, "
                  f"Total Reward: {total_reward:.2f}, Avg Waiting Time: {avg_waiting_time:.2f}s, "
                  f"Avg Emissions: {avg_emissions:.2f}g, Avg Loss: {avg_loss:.4f}, "
                  f"Epsilon: {agent.epsilon:.4f}")
            
            log_file.write(f"Episode Duration: {episode_duration:.2f}s\n")
            log_file.write(f"Total Reward: {total_reward:.2f}\n")
            log_file.write(f"Avg Waiting Time: {avg_waiting_time:.2f}s\n")
            log_file.write(f"Avg Emissions: {avg_emissions:.2f}g\n")
            log_file.write(f"Avg Loss: {avg_loss:.4f}\n")
            log_file.write(f"Epsilon: {agent.epsilon:.4f}\n")
            
    except Exception as e:
        print(f"Training interrupted: {e}")
        log_file.write(f"Training interrupted: {e}\n")
        import traceback
        traceback.print_exc()
        log_file.write(traceback.format_exc())
    
    finally:
        # Close log file
        log_file.write(f"\nSegment completed at: {datetime.datetime.now()}\n")
        log_file.close()
        
        # Make sure SUMO is closed
        try:
            traci.close()
        except:
            pass
    
    return rewards_history, waiting_time_history, emissions_history


In [50]:
def evaluate_agent_segment(agent, sumocfg_path, combined_data, num_evaluation_runs=5):
    """Evaluate the trained agent"""
    print("Starting evaluation...")
    
    # Create evaluation log directory
    eval_log_dir = "logs/evaluation"
    os.makedirs(eval_log_dir, exist_ok=True)
    
    # Log file for evaluation
    log_file = open(f"{eval_log_dir}/evaluation_log.txt", "w")
    log_file.write(f"Evaluation started at: {datetime.datetime.now()}\n")
    log_file.write(f"Number of evaluation runs: {num_evaluation_runs}\n\n")
    
    waiting_times = []
    emissions = []
    rewards = []
    
    try:
        for run in range(num_evaluation_runs):
            print(f"Evaluation run {run+1}/{num_evaluation_runs}")
            log_file.write(f"\nEvaluation run {run+1}/{num_evaluation_runs}\n")
            
            # Connect to SUMO
            try:
                init_sumo(sumocfg_path)
                log_file.write("SUMO initialized successfully\n")
            except Exception as e:
                print(f"Error initializing SUMO: {e}")
                log_file.write(f"Error initializing SUMO: {e}\n")
                continue
            
            # Get traffic light IDs
            traffic_light_ids = traci.trafficlight.getIDList()
            
            # Initialize run variables
            total_reward = 0
            previous_data = None
            run_waiting_times = []
            run_emissions = []
            
            # Run simulation with trained agent
            for step in range(3700):  # Use the same number of steps as in your EDA
                if step % 100 == 0:
                    print(f"Evaluation run {run+1}, Step {step}/3700")
                
                current_timestep = step
                
                # Get current state
                current_data = combined_data[combined_data["Timestep"] == current_timestep]
                if current_data.empty:
                    print(f"No data for timestep {current_timestep}, ending run")
                    log_file.write(f"No data for timestep {current_timestep}, ending run\n")
                    break
                    
                state = get_state(combined_data, current_timestep)
                if state is None:
                    print(f"Could not get state for timestep {current_timestep}, skipping step")
                    log_file.write(f"Could not get state for timestep {current_timestep}, skipping step\n")
                    continue
                
                # Select action (no exploration during evaluation)
                with torch.no_grad():
                    state_tensor = torch.FloatTensor(state).unsqueeze(0)
                    q_values = agent.q_network(state_tensor)
                    action = q_values.argmax().item()
                
                # Take action in environment
                action_success = take_action(action, traffic_light_ids)
                if not action_success:
                    print("Action failed, skipping step")
                    log_file.write("Action failed, skipping step\n")
                    continue
                
                # Advance simulation
                traci.simulationStep()
                
                # Calculate reward (for logging purposes)
                reward = calculate_reward(current_data, previous_data, action)
                total_reward += reward
                
                # Record metrics
                current_waiting_time = current_data["Waiting_Time"].mean()
                current_emissions = current_data["CO2_Emission"].mean()
                run_waiting_times.append(current_waiting_time)
                run_emissions.append(current_emissions)
                
                # Update previous data
                previous_data = current_data
            
            # Close SUMO connection
            traci.close()
            
            # Calculate run metrics
            avg_waiting_time = np.mean(run_waiting_times) if run_waiting_times else 0
            avg_emissions = np.mean(run_emissions) if run_emissions else 0
            
            # Collect metrics
            waiting_times.append(avg_waiting_time)
            emissions.append(avg_emissions)
            rewards.append(total_reward)
            
            # Log run results
            log_file.write(f"Total Reward: {total_reward:.2f}\n")
            log_file.write(f"Average Waiting Time: {avg_waiting_time:.2f}s\n")
            log_file.write(f"Average CO2 Emissions: {avg_emissions:.2f}g\n")
        
        # Calculate overall metrics
        avg_waiting_time = np.mean(waiting_times)
        avg_emissions = np.mean(emissions)
        avg_reward = np.mean(rewards)
        
        # Log overall results
        log_file.write(f"\nOverall Results:\n")
        log_file.write(f"Average Reward: {avg_reward:.2f}\n")
        log_file.write(f"Average Waiting Time: {avg_waiting_time:.2f}s\n")
        log_file.write(f"Average CO2 Emissions: {avg_emissions:.2f}g\n")
        
        print(f"Evaluation Results:")
        print(f"Average Reward: {avg_reward:.2f}")
        print(f"Average Waiting Time: {avg_waiting_time:.2f}s")
        print(f"Average CO2 Emissions: {avg_emissions:.2f}g")
        
    except Exception as e:
        print(f"Evaluation interrupted: {e}")
        log_file.write(f"Evaluation interrupted: {e}\n")
        import traceback
        traceback.print_exc()
        log_file.write(traceback.format_exc())
    
    finally:
        # Close log file
        log_file.write(f"\nEvaluation completed at: {datetime.datetime.now()}\n")
        log_file.close()
        
        # Make sure SUMO is closed
        try:
            traci.close()
        except:
            pass
    
    return avg_waiting_time, avg_emissions, avg_reward

In [51]:
def plot_segment_results(segment, rewards, waiting_times, emissions):
    """Plot and save results for a training segment"""
    plt.figure(figsize=(15, 15))
    
    # Plot rewards
    plt.subplot(3, 1, 1)
    plt.plot(rewards, 'b-')
    plt.title(f'Rewards (Segment {segment})')
    plt.xlabel('Episode')
    plt.ylabel('Total Reward')
    plt.grid(True)
    
    # Plot waiting times
    plt.subplot(3, 1, 2)
    plt.plot(waiting_times, 'r-')
    plt.title('Average Waiting Time')
    plt.xlabel('Episode')
    plt.ylabel('Waiting Time (s)')
    plt.grid(True)
    
    # Plot emissions
    plt.subplot(3, 1, 3)
    plt.plot(emissions, 'g-')
    plt.title('Average CO2 Emissions')
    plt.xlabel('Episode')
    plt.ylabel('CO2 (g)')
    plt.grid(True)
    
    plt.tight_layout()
    plt.savefig(f'results/segment_{segment}_results.png')
    plt.close()

In [52]:
def main_segmented():
    """Main function with segmented training to allow for recovery"""
    print("Starting traffic light optimization with RL...")
    
    # Create necessary directories
    os.makedirs("checkpoints", exist_ok=True)
    os.makedirs("results", exist_ok=True)
    os.makedirs("models", exist_ok=True)
    os.makedirs("logs", exist_ok=True)
    
    # Load data
    combined_data = load_data()
    if combined_data is None:
        print("Failed to load data. Exiting.")
        return
    
    # Save configuration
    config = {
        "sumocfg_path": "C:/Users/Pushp Raj Choudhary/2025-03-14-16-24-53/osm.sumocfg",
        "total_episodes": 100,
        "episodes_per_segment": 10,
        "max_steps_per_episode": 3700,
        "checkpoint_interval": 2,
        "learning_rate": 0.001,
        "gamma": 0.95,
        "epsilon_start": 1.0,
        "epsilon_decay": 0.995,
        "epsilon_min": 0.01,
        "buffer_size": 10000,
        "batch_size": 32,
        "evaluation_runs": 5
    }
    
    with open("config.json", "w") as f:
        json.dump(config, f, indent=4)
    
    # Define state and action sizes
    # Adjust state_size based on your actual state representation
    state_size = 50  # This should match the length of your state vector
    action_size = 4  # Do nothing, Switch phase, Extend phase, Reduce phase
    
    # Create agent
    agent = DQNAgent(
        state_size=state_size,
        action_size=action_size,
        learning_rate=config["learning_rate"],
        gamma=config["gamma"],
        epsilon=config["epsilon_start"],
        epsilon_decay=config["epsilon_decay"],
        epsilon_min=config["epsilon_min"],
        buffer_size=config["buffer_size"],
        batch_size=config["batch_size"]
    )
    
    # Training parameters
    total_episodes = config["total_episodes"]
    episodes_per_segment = config["episodes_per_segment"]
    max_steps_per_episode = config["max_steps_per_episode"]
    sumocfg_path = config["sumocfg_path"]
    checkpoint_interval = config["checkpoint_interval"]
    
    # Check for resume flag
    resume_from = None
    resume_segment = 0
    
    # Check if there are existing checkpoints
    checkpoints = [f for f in os.listdir("checkpoints") if f.startswith("agent_checkpoint_ep")]
    if checkpoints:
        # Extract episode numbers from checkpoint filenames
        checkpoint_episodes = [int(f.split("ep")[1].split(".")[0]) for f in checkpoints]
        if checkpoint_episodes:
            latest_checkpoint = max(checkpoint_episodes)
            print(f"Found latest checkpoint at episode {latest_checkpoint}")
            resume_option = input(f"Resume from episode {latest_checkpoint}? (y/n): ")
            if resume_option.lower() == 'y':
                resume_from = latest_checkpoint
                resume_segment = resume_from // episodes_per_segment
                print(f"Resuming from segment {resume_segment}")
    
    # Train in segments
    try:
        for segment in range(resume_segment, total_episodes // episodes_per_segment):
            start_episode = segment * episodes_per_segment
            end_episode = min((segment + 1) * episodes_per_segment, total_episodes)
            
            print(f"\n{'='*50}")
            print(f"Starting training segment {segment+1}/{total_episodes // episodes_per_segment}")
            print(f"Episodes {start_episode} to {end_episode-1}")
            print(f"{'='*50}\n")
            
            # Train for this segment
            rewards, waiting_times, emissions = train_agent_segment(
                agent=agent,
                num_episodes=episodes_per_segment,
                max_steps_per_episode=max_steps_per_episode,
                sumocfg_path=sumocfg_path,
                combined_data=combined_data,
                start_episode=start_episode,
                end_episode=end_episode,
                checkpoint_interval=checkpoint_interval,
                resume_from=resume_from if segment == resume_segment else None
            )
            
            # Only use resume_from for the first segment after resuming
            resume_from = None
            
            # Save segment results
            np.save(f"results/segment_{segment}_rewards.npy", np.array(rewards))
            np.save(f"results/segment_{segment}_waiting_times.npy", np.array(waiting_times))
            np.save(f"results/segment_{segment}_emissions.npy", np.array(emissions))
            
            # Plot segment results
            plot_segment_results(segment, rewards, waiting_times, emissions)
            
            # Save model after each segment
            torch.save(agent.q_network.state_dict(), f"models/q_network_segment_{segment}.pth")
            
            # Evaluate after each segment
            print("\nEvaluating agent after segment completion...")
            avg_waiting, avg_emissions, avg_reward = evaluate_agent_segment(
                agent=agent,
                sumocfg_path=sumocfg_path,
                combined_data=combined_data,
                num_evaluation_runs=config["evaluation_runs"]
            )
            
            # Save evaluation results
            with open(f"results/segment_{segment}_evaluation.json", "w") as f:
                json.dump({
                    "avg_waiting_time": avg_waiting,
                    "avg_emissions": avg_emissions,
                    "avg_reward": avg_reward
                }, f, indent=4)
            
            # Ask if user wants to continue to next segment
            if segment < (total_episodes // episodes_per_segment) - 1:
                continue_option = input(f"Continue to segment {segment+2}? (y/n): ")
                if continue_option.lower() != 'y':
                    print("Training paused. You can resume later from the latest checkpoint.")
                    break
    
    except KeyboardInterrupt:
        print("\nTraining interrupted by user.")
        print("You can resume later from the latest checkpoint.")
    
    except Exception as e:
        print(f"\nError during training: {e}")
        import traceback
        traceback.print_exc()
        print("You can resume later from the latest checkpoint.")
    
    finally:
        # Final evaluation after all segments
        print("\nPerforming final evaluation...")
        final_waiting, final_emissions, final_reward = evaluate_agent_segment(
            agent=agent,
            sumocfg_path=sumocfg_path,
            combined_data=combined_data,
            num_evaluation_runs=config["evaluation_runs"] * 2  # More runs for final evaluation
        )
        
        # Save final model
        torch.save(agent.q_network.state_dict(), "models/q_network_final.pth")
        
        # Save final evaluation results
        with open("results/final_evaluation.json", "w") as f:
            json.dump({
                "avg_waiting_time": final_waiting,
                "avg_emissions": final_emissions,
                "avg_reward": final_reward
            }, f, indent=4)
        
        print("\nTraining completed!")
        print(f"Final model saved to models/q_network_final.pth")
        print(f"Final evaluation results: Avg Waiting Time: {final_waiting:.2f}s, Avg Emissions: {final_emissions:.2f}g")

In [55]:
if __name__ == "__main__":
    # Set random seeds for reproducibility
    random.seed(42)
    np.random.seed(42)
    torch.manual_seed(42)
    
    # Add current date and time to log files
    current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    os.makedirs(f"logs/{current_time}", exist_ok=True)
    
    # Redirect stdout to both console and log file
    log_file = open(f"logs/{current_time}/main_log.txt", "w")
    
    class Logger:
        def __init__(self, console, file):
            self.console = console
            self.file = file
            
        def write(self, message):
            self.console.write(message)
            self.file.write(message)
            
        def flush(self):
            self.console.flush()
            self.file.flush()
    
    sys.stdout = Logger(sys.stdout, log_file)
    
    # Print start time
    print(f"Starting at: {current_time}")
    print(f"Python version: {sys.version}")
    print(f"PyTorch version: {torch.__version__}")
    
    # Run the main function
    main_segmented()
    
    # Close log file
    log_file.close()

KeyboardInterrupt: 